In [1]:
import io
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from psycopg2 import connect
from sqlalchemy import create_engine

# Creates the initial table for items purchased
# It is only needed for loading data for the first time
csv = "../data/items_purchased.csv"
#csv = "~/Desktop/items_purchased.csv"

def writeToPostgres(f):
    #conn = connect("postgresql://tlapinskas@localhost/postgres")
    conn = connect("postgresql://INSERT-USERNAME@localhost/postgres")
    cursor = conn.cursor()
    print("Creating table...\n")
    cursor.execute('create table items_purchased (kiosk_id text, product_id text, card_hash text, date_time text, fc_number text);COMMIT; ')
    cursor.copy_from(f, 'items_purchased', columns=('kiosk_id', 'product_id', 'card_hash', 'date_time', 'fc_number'), sep='\t')
    cursor.close()

# Read columns
df = pd.read_csv(csv, usecols = ['kiosk_id', 'product_id', 'card_hash', 'date_time', 'fc_number'])

# Load data to temp CSV
f = io.StringIO()
df.to_csv(f, index=False, header=False, sep='\t')
f.seek(0)

# Load to Postgres
writeToPostgres(f)

print("Table created and CSV load complete")

/usr/local/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning:

The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.



Creating table...

Table created and CSV load complete


In [2]:
# Loads data incrementally if format is the same as items_purchased.csv
# Use case: Hourly/Daily/Weekly load of items purchased data that needs to live in a Data Lake
csv = "../data/items_purchased.csv"
#csv = "/Users/tlapinskas/Desktop/items_purchased.csv"

def loadToPostgres(csv):
    #conn = connect("postgresql://tlapinskas@localhost/postgres")
    conn = connect("postgresql://INSERT-USERNAME@localhost/postgres")
    cur = conn.cursor()
    print("Loading more data...\n")
    with open(csv, 'r') as f:
        next(f)  # Skip the header row.
        cur.copy_from(f, 'items_purchased', sep=',')

    conn.commit()

    cur.execute ('''SELECT COUNT(*) FROM items_purchased;''')
    count = cur.fetchone()
    count = str(count)
    count = count.replace('(', '').replace(')', '').replace(',', '')
    
    return count

count = loadToPostgres(csv)

print("Data load complete")
print ("Total records in items_purchased table:", count)

Loading more data...

Data load complete
Total records in items_purchased table: 2231144


In [3]:
# Creates the initial table for product information csv
# It is only needed for loading data for the first time
csv = "../data/product_info.csv"

def writeToPostgres(f):
    #conn = connect("postgresql://tlapinskas@localhost/postgres")
    conn = connect("postgresql://INSERT-USERNAME@localhost/postgres")
    cursor = conn.cursor()
    print("Creating table...\n")
    cursor.execute('create table product_info (product_id text, product_name text, fc_number text, brand text, onqi text, cost text, margins text);COMMIT; ')
    cursor.copy_from(f, 'product_info', columns=('product_id', 'product_name', 'fc_number', 'brand', 'onqi', 'cost', 'margins'), sep='\t')
    cursor.close()

# Read columns
df = pd.read_csv(csv, usecols = ['product_id', 'product_name', 'fc_number', 'brand', 'onqi', 'cost', 'margins'])

# Load data to temp CSV
f = io.StringIO()
df.to_csv(f, index=False, header=False, sep='\t')
f.seek(0)

# Load to Postgres
writeToPostgres(f)

print("Table created and CSV load complete")

Creating table...

Table created and CSV load complete


In [4]:
# Loads data incrementally if format is the same as product_info.csv
# Use case: Hourly/Daily/Weekly load of items purchased data that needs to live in a Data Lake
csv = "../data/product_info.csv"

def loadToPostgres(csv):
    #conn = connect("postgresql://tlapinskas@localhost/postgres")
    conn = connect("postgresql://INSERT-USERNAME@localhost/postgres")
    cur = conn.cursor()
    print("Loading more data...\n")
    with open(csv, 'r') as f:
        next(f)  # Skip the header row.
        cur.copy_from(f, 'product_info', sep=',')

    conn.commit()

    cur.execute ('''SELECT COUNT(*) FROM product_info;''')
    count = cur.fetchone()
    count = str(count)
    count = count.replace('(', '').replace(')', '').replace(',', '')
    
    return count

count = loadToPostgres(csv)

print("Data load complete")
print ("Total records in product_info table:", count)

Loading more data...

Data load complete
Total records in product_info table: 660


In [5]:
init_notebook_mode(connected=True)    

#engine = create_engine("postgresql://tlapinskas@localhost/postgres")
engine = create_engine("postgresql://INSERT-USERNAME@localhost/postgres")

# Grab data from Postgres
df = pd.read_sql('''
                    SELECT p.product_id, p.card_hash, pi.onqi
                    FROM items_purchased p, product_info pi
                    WHERE p.product_id = pi.product_id AND onqi > '50'
                    LIMIT 100;
                    ''', con=engine)
df = df.fillna('')

trace1 = go.Scatter(
    x=df['product_id'],
    y=df['onqi'],
    text=df['card_hash'],
    mode='markers'
)

layout = go.Layout(
    title='ONQI versus Card_Hash & Product_ID',
    xaxis=dict( type='log', title='PRODUCT_ID' ),
    yaxis=dict( title='ONQI' )
)

data = [trace1]
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='test')